In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM

Using TensorFlow backend.


In [0]:
data = pd.read_csv("drive/My Drive/JS_peaks1_5.csv", index_col= "DateTime")
data.index = pd.to_datetime(data.index)

In [0]:
df = data.drop(["high","low","open","volume"],axis=1)
#df.peaks.replace(-1,2,inplace=True)
#df = df.drop("volume",axis=1)
#df = df[-10250:]

In [0]:
buy = 0
sell = 0
idle = 0
imagesX = []
imagesY = []
dfX = df.drop("peaks",axis=1).values
peaks = df.peaks.values
for i in range(1,len(dfX)):
  if(peaks[i] == 1 and sell<=buy):
    imagesX.append(dfX[i-1:i+1])
    imagesY.append(peaks[i])
    sell += 1
  if(peaks[i] == -1 and buy<=sell):
    imagesX.append(dfX[i-1:i+1])
    imagesY.append(peaks[i])
    buy += 1
  elif(idle<buy):
    imagesX.append(dfX[i-1:i+1])
    imagesY.append(peaks[i])
    idle += 1

In [18]:
print("Buy",buy)
print("Sell",sell)
print("Idle",idle)
print("Shape",np.array(imagesX).shape)

Buy 20683
Sell 20683
Idle 20683
Shape (62049, 2, 25)


In [0]:
batch_size = 64
num_classes = 3
epochs = 10

img_rows, img_cols = 2, 25
size = 61000
input_shape = (img_rows, img_cols, 1)

x_train0 = imagesX[:size]
x_test0 = imagesX[size:]

y_train0 = imagesY[:size]
y_test0 = imagesY[size:]

x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train0, y_train0, train_size=0.8, random_state=116)

x_train1 = np.array(x_train1)
x_train = x_train1.reshape(x_train1.shape[0], img_rows, img_cols, 1)

x_test1 = np.array(x_test1)
x_test = x_test1.reshape(x_test1.shape[0], img_rows, img_cols, 1)


y_train = keras.utils.to_categorical(y_train1, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test1, num_classes=num_classes)



In [0]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', input_shape=input_shape))
model.add(Conv2D(256, (1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.35))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.50))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['categorical_accuracy'])

In [33]:
model.fit(x_train, y_train,
          batch_size=32, 
          epochs=100, 
          verbose=1,
          validation_data=(x_test, y_test))

Train on 48800 samples, validate on 12200 samples
Epoch 1/100
48800/48800 [==============================] - 83s 2ms/step - loss: 10.1939 - categorical_accuracy: 0.3675 - val_loss: 10.1848 - val_categorical_accuracy: 0.3681
Epoch 2/100
48800/48800 [==============================] - 79s 2ms/step - loss: 10.1518 - categorical_accuracy: 0.3701 - val_loss: 10.1848 - val_categorical_accuracy: 0.3681
Epoch 3/100
48800/48800 [==============================] - 86s 2ms/step - loss: 9.4498 - categorical_accuracy: 0.4106 - val_loss: 5.9509 - val_categorical_accuracy: 0.6281
Epoch 4/100
48800/48800 [==============================] - 86s 2ms/step - loss: 4.7866 - categorical_accuracy: 0.6966 - val_loss: 3.1840 - val_categorical_accuracy: 0.7959
Epoch 5/100
48800/48800 [==============================] - 85s 2ms/step - loss: 3.5990 - categorical_accuracy: 0.7717 - val_loss: 2.5092 - val_categorical_accuracy: 0.8402
Epoch 6/100
48800/48800 [==============================] - 86s 2ms/step - loss: 2.6765

In [34]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 2, 25, 128)        256       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 2, 25, 256)        33024     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 25, 256)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 2, 25, 256)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1638528   
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)              